In [10]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [11]:
DATA_PATH = "../data/processed/financial_ratios_final_clean.csv"
OUT_DIR = "../outputs/tables"
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(DATA_PATH, encoding="utf-8-sig")
print("Loaded:", df.shape)

Loaded: (1469, 20)


In [12]:
firms_by_industry = (
    df.groupby("Ngành ICB - cấp 1")["Mã"]
      .nunique()
      .sort_values(ascending=False)
)
display(firms_by_industry)

Ngành ICB - cấp 1
Công nghiệp            100
Hàng Tiêu dùng          64
Nguyên vật liệu         59
Tiện ích Cộng đồng      37
Dịch vụ Tiêu dùng       23
Dược phẩm và Y tế       15
Công nghệ Thông tin      8
Dầu khí                  3
Name: Mã, dtype: int64

In [13]:
RATIO_GROUPS = {
    "Liquidity": [
        "Current_Ratio",
        "Quick_Ratio",
        "Cash_Ratio"
    ],
    "Leverage": [
        "Debt_Equity",
        "Net_Leverage"
    ],
    "Efficiency": [
        "Asset_Turnover",
        "Fixed_Asset_Turnover"
    ],
    "Profitability": [
        "ROA",
        "ROE",
        "ROS"
    ]
}

In [16]:
def run_pca_block(df_block, ratio_cols, explained_threshold=0.8, min_obs=2):
    """
    Run PCA for a block (industry-year) within one ratio group.
    Returns:
    - explained variance ratios
    - cumulative explained variance
    - number of PCs needed to reach explained_threshold
    - loadings matrix
    """
    X = df_block[ratio_cols].dropna()
    if X.shape[0] < min_obs:
        return None  # not enough observations

    X_scaled = StandardScaler().fit_transform(X)

    # Ensure n_components is valid even when n_obs is small
    n_comp = min(X_scaled.shape[0], X_scaled.shape[1])
    if n_comp < 1:
        return None

    pca = PCA(n_components=n_comp)
    pca.fit(X_scaled)

    explained = pca.explained_variance_ratio_
    cum_explained = np.cumsum(explained)

    n_components = int(np.argmax(cum_explained >= explained_threshold) + 1)

    # IMPORTANT FIX: columns must match the actual number of PCs
    loadings = pd.DataFrame(
        pca.components_.T,
        index=ratio_cols,
        columns=[f"PC{i+1}" for i in range(pca.components_.shape[0])]
    )

    return {
        "n_obs": X.shape[0],
        "explained_variance": explained,
        "cum_explained": cum_explained,
        "n_components_80pct": n_components,
        "loadings": loadings
    }

In [17]:
records = []
representative_indicators = []

for (industry, year), df_sub in df.groupby(["Ngành ICB - cấp 1", "Năm"]):

    for group_name, cols in RATIO_GROUPS.items():

        result = run_pca_block(df_sub, cols, explained_threshold=0.8, min_obs=2)

        if result is None:
            # Keep the industry-year-group row (do NOT drop the industry)
            records.append({
                "Ngành ICB - cấp 1": industry,
                "Năm": year,
                "Nhóm chỉ số": group_name,
                "Số quan sát": df_sub[cols].dropna().shape[0],
                "Số PC đạt 80% thông tin": np.nan,
                "Tỷ lệ phương sai PC1": np.nan
            })
            representative_indicators.append({
                "Ngành ICB - cấp 1": industry,
                "Năm": year,
                "Nhóm chỉ số": group_name,
                "Chỉ số đại diện (theo PCA)": np.nan
            })
            continue

        pc1_loadings = result["loadings"]["PC1"].abs()
        rep_indicator = pc1_loadings.idxmax()

        records.append({
            "Ngành ICB - cấp 1": industry,
            "Năm": year,
            "Nhóm chỉ số": group_name,
            "Số quan sát": result["n_obs"],
            "Số PC đạt 80% thông tin": result["n_components_80pct"],
            "Tỷ lệ phương sai PC1": round(float(result["explained_variance"][0]), 3)
        })

        representative_indicators.append({
            "Ngành ICB - cấp 1": industry,
            "Năm": year,
            "Nhóm chỉ số": group_name,
            "Chỉ số đại diện (theo PCA)": rep_indicator
        })


In [18]:
pca_summary_df = pd.DataFrame(records).sort_values(
    ["Ngành ICB - cấp 1", "Năm", "Nhóm chỉ số"]
)
pca_summary_path = os.path.join(OUT_DIR, "pca_structure_by_industry_year.csv")
pca_summary_df.to_csv(pca_summary_path, index=False, encoding="utf-8-sig")
print("Saved:", pca_summary_path)

Saved: ../outputs/tables/pca_structure_by_industry_year.csv


In [19]:
rep_indicator_df = pd.DataFrame(representative_indicators).sort_values(
    ["Ngành ICB - cấp 1", "Năm", "Nhóm chỉ số"]
)
rep_path = os.path.join(OUT_DIR, "representative_indicators_by_industry_year.csv")
rep_indicator_df.to_csv(rep_path, index=False, encoding="utf-8-sig")
print("Saved:", rep_path)

Saved: ../outputs/tables/representative_indicators_by_industry_year.csv
